In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

In [22]:
DATA_PATH = "data/data.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,price
0,2014-05-02 00:00:00,3.0,1.50,1340,7912,1.5,0,0,3.0,1340,0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,313000.0
1,2014-05-02 00:00:00,5.0,2.50,3650,9050,2.0,0,4,5.0,3370,280,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA,2384000.0
2,2014-05-02 00:00:00,3.0,2.00,1930,11947,1.0,0,0,4.0,1930,0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,342000.0
3,2014-05-02 00:00:00,3.0,2.25,2000,8030,1.0,0,0,4.0,1000,1000,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA,420000.0
4,2014-05-02 00:00:00,4.0,2.50,1940,10500,1.0,0,0,4.0,1140,800,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA,550000.0


In [23]:
# Выводим структурные сведенья о DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   bedrooms       4600 non-null   float64
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   int64  
 4   sqft_lot       4600 non-null   int64  
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   int64  
 7   view           4600 non-null   int64  
 8   condition      4520 non-null   float64
 9   sqft_above     4600 non-null   int64  
 10  sqft_basement  4600 non-null   int64  
 11  yr_built       4390 non-null   float64
 12  yr_renovated   4600 non-null   int64  
 13  street         4600 non-null   object 
 14  city           4589 non-null   object 
 15  statezip       4600 non-null   object 
 16  country        4600 non-null   object 
 17  price          4600 non-null   float64
dtypes: float

Структура следующая:
date – дата продажи
bedrooms – количество спален
bathrooms – количество ванных комнат
sqft_living – жилая площадь
sqft_lot – общая площадь участка
floors – количество этажей
waterfront – наличие выхода к воде
view – оценка вида
condition – оценка состояния дома
sqft_above – площадь над уровнем земли
sqft_basement – площадь под уровнем земли
yr_built – год постройки дома
yr_renovated – год последнего ремонта
street – название улицы
city – город
statezip – комбинация штата и ZIP-кода
country – страна

In [24]:
# Выводит статистику по столбцам
df.describe(include='all')


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,price
count,4600,4600.000000,4600.000000,4600.000000,4.600000e+03,4600.000000,4600.000000,4600.000000,4520.000000,4600.000000,4600.000000,4390.000000,4600.000000,4600,4589,4600,4600,4.600000e+03
unique,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4525,44,77,1,NaN
top,2014-06-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2520 Mulberry Walk NE,Seattle,WA 98103,USA,NaN
freq,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1571,148,4600,NaN
mean,NaN,3.400870,2.160815,2139.346957,1.485252e+04,1.512065,0.007174,0.240652,3.450664,1827.265435,312.081522,1970.828929,808.608261,NaN,NaN,NaN,NaN,5.519630e+05
std,NaN,0.908848,0.783781,963.206916,3.588444e+04,0.538288,0.084404,0.778405,0.678601,862.168977,464.137228,29.736162,979.414536,NaN,NaN,NaN,NaN,5.638347e+05
min,NaN,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,3.000000,1.750000,1460.000000,5.000750e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000,NaN,NaN,NaN,NaN,3.228750e+05
50%,NaN,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000,NaN,NaN,NaN,NaN,4.609435e+05
75%,NaN,4.000000,2.500000,2620.000000,1.100125e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000,NaN,NaN,NaN,NaN,6.549625e+05


Числовые признаки:
count - количество не пустых значений,
mean - среднее арифметическое,
std - стандартное отклонение,
min, 25%, 50%, 75%, max - значения.

Категориальные/строковые признаки:
unique - количество уникальных элементов,
top - самое частое значение,
freq - частота самого частого значения.

Заметки по данным:
date – строковый признак, требует преобразования в datetime (преобразуем в строку, можно будет использовать вместе с yr_built, месяц возможно полезен, но не точно)
bedrooms – числовой признак, распределён нормально (оставляем как есть)
bathrooms – числовой признак, распределён нормально (оставляем как есть)
sqft_living – числовой признак, скошено вправо (применяем log)
sqft_lot – числовой признак, сильно скошено вправо (применяем log)
floors – дискретный числовой признак, нормальная форма (оставляем как есть)
waterfront – бинарный признак 0/1 (оставляем как есть)
view – дискретный рейтинговый признак 0–4 (оставляем как есть)
condition – дискретный рейтинговый признак 1–5 (оставляем как есть)
sqft_above – числовой признак, скошено вправо (применяем log)
sqft_basement – числовой признак, скошено вправо (применяем log)
yr_built – числовой признак, монотонный диапазон (преобразуем в возраст)
yr_renovated – числовой признак, почти всегда 0 (преобразуем в бинарный ремонт)
street – категориальный признак высокой кардинальности, значения почти уникальны (не используем)
city – категориальный признак высокой кардинальности (не используем)
statezip – категориальный признак высокой кардинальности (не используем)
country – константа, одно значение (не используем)

In [25]:
# Показывает количество пропусков в каждом столбце
df.isna().sum()

date               0
bedrooms           0
bathrooms          0
sqft_living        0
sqft_lot           0
floors             0
waterfront         0
view               0
condition         80
sqft_above         0
sqft_basement      0
yr_built         210
yr_renovated       0
street             0
city              11
statezip           0
country            0
price              0
dtype: int64

condition - 80 пропусков (можно заполнить медианой)
yr_built - 210 пропусков (можно заполнить медианой)
city - 11 пропусков (уже не используем)


In [26]:
# ПРЕДОБРАБОТКА

# Преобразовываем дату
df['date'] = pd.to_datetime(df['date'])

df['sale_year'] = df['date'].dt.year
df['sale_month'] = df['date'].dt.month

df = df.drop(columns=['date'])

# Обрабатываем пропуски
df['condition'] = df['condition'].fillna(df['condition'].median())
df['yr_built'] = df['yr_built'].fillna(df['yr_built'].median())
df['city'] = df['city'].fillna('unknown')

# Более удобные признаки
df['house_age'] = df['sale_year'] - df['yr_built']
df = df.drop(columns=['sale_year'])
df['renovated'] = (df['yr_renovated'] > 0).astype(int)
df = df.drop(columns=['yr_built', 'yr_renovated'])

# Удаляем не используемые признаки
df = df.drop(columns=['street', 'statezip','city','country'])

# Логарифмируем (log1p = log(x+1), безопасная операция с нулями)
for col in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']:
    df[col] = np.log1p(df[col])


In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       4600 non-null   float64
 1   bathrooms      4600 non-null   float64
 2   sqft_living    4600 non-null   float64
 3   sqft_lot       4600 non-null   float64
 4   floors         4600 non-null   float64
 5   waterfront     4600 non-null   int64  
 6   view           4600 non-null   int64  
 7   condition      4600 non-null   float64
 8   sqft_above     4600 non-null   float64
 9   sqft_basement  4600 non-null   float64
 10  price          4600 non-null   float64
 11  sale_month     4600 non-null   int32  
 12  house_age      4600 non-null   float64
 13  renovated      4600 non-null   int64  
dtypes: float64(10), int32(1), int64(3)
memory usage: 485.3 KB


In [28]:
df.describe()


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,price,sale_month,house_age,renovated
count,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4.600000e+03,4600.000000,4600.000000,4600.000000
mean,3.400870,2.160815,7.576934,9.002273,1.512065,0.007174,0.240652,3.442826,7.413444,2.616127,5.519630e+05,5.757609,42.935000,0.405435
std,0.908848,0.783781,0.429669,0.910537,0.538288,0.084404,0.778405,0.675249,0.435781,3.207574,5.638347e+05,0.683851,29.069372,0.491029
min,0.000000,0.000000,5.916202,6.459904,1.000000,0.000000,0.000000,1.000000,5.916202,0.000000,0.000000e+00,5.000000,0.000000,0.000000
25%,3.000000,1.750000,7.286876,8.517543,1.000000,0.000000,0.000000,3.000000,7.082549,0.000000,3.228750e+05,5.000000,18.000000,0.000000
50%,3.000000,2.250000,7.591357,8.946896,1.500000,0.000000,0.000000,3.000000,7.372118,0.000000,4.609435e+05,6.000000,38.000000,0.000000
75%,4.000000,2.500000,7.871311,9.305855,2.000000,0.000000,0.000000,4.000000,7.741099,6.415097,6.549625e+05,6.000000,62.000000,1.000000
max,9.000000,8.000000,9.513477,13.887104,3.500000,1.000000,4.000000,5.000000,9.149634,8.480737,2.659000e+07,7.000000,114.000000,1.000000


In [29]:
y = df['price']
X = df.drop(columns=['price'])

In [30]:
# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [31]:
# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

In [32]:
from sklearn.metrics import r2_score, mean_squared_error


print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)



R2: 0.03315188955501036
RMSE: 992993.7093744905


Пока всё грустно и печально))

In [33]:
# работаем с уже предобработанным df (до разбиения)
corr = df.corr(numeric_only=True)['price'].sort_values(ascending=False)
corr


price            1.000000
sqft_living      0.378632
sqft_above       0.335074
bathrooms        0.327110
view             0.228504
bedrooms         0.200336
floors           0.151461
waterfront       0.135648
sqft_basement    0.128617
sqft_lot         0.098131
sale_month       0.041081
condition        0.031503
house_age       -0.024408
renovated       -0.028595
Name: price, dtype: float64

In [34]:
df['sqft_living_sq'] = df['sqft_living'] ** 2
df['bath_bed_ratio'] = df['bathrooms'] / df['bedrooms']
df = df.dropna(subset=['bath_bed_ratio'])

In [35]:
y = df['price']
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)


R2: 0.060976078773375075
RMSE: 903147.4834665706
